In [1]:
from keras.preprocessing import image

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import *

In [3]:
from keras import applications

In [14]:
from tensorflow.keras.applications.vgg16 import VGG16

In [5]:
from keras import Model

In [15]:
base_model= VGG16(weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58900480/58889256 [==============================] - 50s 1us/step


#FREEZING THE MODEL

In [16]:
base_model.trainable = False

#ADDING INPUT ,POOLING,OUTPUT LAYERS

In [17]:
inputs = Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = GlobalAveragePooling2D()(x)
d1 = Dense(256, activation = 'relu')(x)
d2=Dropout(rate=(0.5))(d1)
d3 = Dense(64, activation = 'relu')(d2)
# Add final dense layer
outputs = Dense(4, activation = 'softmax')(d3)

# Combine inputs and outputs to create model
model = Model(inputs,outputs)

In [18]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                16448     
                                                           

In [19]:
model.compile(loss = "categorical_crossentropy" , metrics = ["accuracy"])

In [ ]:
import os
import shutil

In [ ]:
if not os.path.isdir('../desktop/val_images'):
    os.mkdir('../desktop/val_images')
    

In [ ]:
classes=['cats','dogs','horses','humans']

In [ ]:
for c in classes:
    p=os.path.join('../desktop/val_images',c)
    if not os.path.isdir(p):
        os.mkdir(p)
split=0.9
for f in os.listdir('../desktop/images'):
    path='../desktop/images/'+f
    img=os.listdir(path)
    split_size=int(split*len(img))
    files_to_move=img[split_size:]
    for val_img in files_to_move:
        s=os.path.join(path,val_img)
        d=os.path.join('../desktop/val_images/'+f,val_img)
        shutil.move(s,d)
    

In [20]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    samplewise_center=True,  
        rotation_range=10,  
        zoom_range = 0.1,
       shear_range=0.2,
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=False)

In [21]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('../desktop/images', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical",
                                    batch_size=32)
# load and iterate validation dataset
val_gen=ImageDataGenerator(rescale=1.0/255)
valid_it=val_gen.flow_from_directory(
'../desktop/val_images',
target_size=(224,224),
class_mode='categorical',
batch_size=32
)

Found 724 images belonging to 4 classes.
Found 84 images belonging to 4 classes.


In [22]:
model.fit(train_it,epochs=20,validation_data=valid_it)

Epoch 1/20
23/23 [==============================] - 46s 2s/step - loss: 1.2713 - accuracy: 0.4157 - val_loss: 0.9726 - val_accuracy: 0.6548
Epoch 2/20
23/23 [==============================] - 45s 2s/step - loss: 0.9195 - accuracy: 0.6298 - val_loss: 0.6068 - val_accuracy: 0.8214
Epoch 3/20
23/23 [==============================] - 45s 2s/step - loss: 0.7413 - accuracy: 0.7030 - val_loss: 0.4989 - val_accuracy: 0.8571
Epoch 4/20
23/23 [==============================] - 45s 2s/step - loss: 0.6246 - accuracy: 0.7610 - val_loss: 0.3867 - val_accuracy: 0.8571
Epoch 5/20
23/23 [==============================] - 45s 2s/step - loss: 0.5210 - accuracy: 0.8052 - val_loss: 0.3231 - val_accuracy: 0.9048
Epoch 6/20
23/23 [==============================] - 44s 2s/step - loss: 0.4988 - accuracy: 0.8149 - val_loss: 0.3220 - val_accuracy: 0.9167
Epoch 7/20
23/23 [==============================] - 45s 2s/step - loss: 0.4420 - accuracy: 0.8232 - val_loss: 0.3262 - val_accuracy: 0.9286
Epoch 8/20
23/23 [==

#FINE TUNING

In [33]:
from tensorflow.keras.optimizers import Adam

In [34]:
# Unfreeze the base model
base_model.trainable = True
adam=Adam(learning_rate = 0.00001)
# Compile the model with a low learning rate
model.compile(optimizer=adam
          , loss = "categorical_crossentropy", metrics = ["accuracy"])

In [35]:
model.fit(train_it,
          validation_data=valid_it,
          epochs=5)

Epoch 1/5
23/23 [==============================] - 201s 9s/step - loss: 0.0845 - accuracy: 0.9682 - val_loss: 0.0562 - val_accuracy: 0.9643
Epoch 2/5
23/23 [==============================] - 198s 9s/step - loss: 0.0543 - accuracy: 0.9765 - val_loss: 0.0425 - val_accuracy: 0.9762
Epoch 3/5
23/23 [==============================] - 236s 10s/step - loss: 0.0312 - accuracy: 0.9862 - val_loss: 0.0241 - val_accuracy: 0.9881
Epoch 4/5
23/23 [==============================] - 382s 17s/step - loss: 0.0286 - accuracy: 0.9890 - val_loss: 0.0271 - val_accuracy: 0.9881
Epoch 5/5
23/23 [==============================] - 371s 16s/step - loss: 0.0203 - accuracy: 0.9959 - val_loss: 0.0319 - val_accuracy: 0.9881


#WE CAN HERE , TRAINING ACCURACY IS ARROUND 99.6% WITH VALIDATION SCORE OF 98.81%!!!

In [36]:
model.save('VGG16_IMAGE_CLASSIFICATION_MODEL')

INFO:tensorflow:Assets written to: VGG16_IMAGE_CLASSIFICATION_MODEL\assets
